# Landuse classification with Sentinel2 satellite images using deep learning

## Introduction
In this execise we will make a landuse classification using satellite images of the bands Blue (02), Green (B03), Red (B04 and Infrared (B08) with a resolution of 10 meter of the [Sentinel-2 satellite](sentinel.esa.int/web/sentinel/missions/sentinel-2). 

If you are not familiar with satellite remote sensing, you can find an introduction [here](https://semiautomaticclassificationmanual-v5.readthedocs.io/de/latest/remote_sensing.html).

We will be using a [deep learning algorithm](https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/) The deep learning algorithm is provided by the `Keras` library. `Keras` is a powerful and easy-to-use free open source Python library for developing and evaluating deep learning models.

It is part of the TensorFlow library and allows you to define and train neural network models in just a few lines of code.
In this tutorial, you will discover how to create your first deep learning neural network model in Python using Keras.

An excellent introduction on deep learning and neural networks can be found [here](https://www.doc.ic.ac.uk/~nuric/teaching/imperial-college-machine-learning-neural-networks.html).

In [7]:

# tensorflow
import tensorflow
print('tensorflow: %s' % tensorflow.__version__)

# keras
import keras
from keras.models import Sequential
from keras.layers import Dense
print('keras: %s' % keras.__version__)

# pandas
import pandas as pd



tensorflow: 2.12.0
keras: 2.12.0


Load data

In [8]:
# https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

# Set dataset to use
WAALWIJK = True

if not WAALWIJK :
    
    # Load the dataset
    df_in = pd.read_csv('data_files\landuse_classification.csv')

    # Mapping of landuse to number
    landuse_classes = {}
    landuse_classes[' waterloop'] = 0
    landuse_classes[' dodenakker'] = 1
    landuse_classes[' zand'] = 2
    landuse_classes[' spoorbaanlichaam'] = 3
    landuse_classes[' Kassengebied'] = 4
    landuse_classes[' heide'] = 5
    landuse_classes[' overig'] = 6
    landuse_classes[' akkerland'] = 7
    landuse_classes[' bos: gemengd bos'] = 8
    landuse_classes[' bebouwd gebied'] = 9
    landuse_classes[' grasland'] = 10

    # Landuses to select
    selected_landuses = [0,1,2,3,4,5,6,7,8,9,10]

    # Remap the values of the dataframe
    df_in["Gewas"] = df_in["Gewas"].apply(lambda x: landuse_classes.get(x))

    # Convert to numeric
    df_in["Gewas"] = pd.to_numeric(df_in["Gewas"])

    # Select landuse classes
    #df = df_in[df_in["Gewas"].isin(selected_landuses)]
    df = df_in[df_in["Gewas"].isin(selected_landuses)]

else:

    df_in = pd.read_csv('data_files\landuse_classification_waalwijk.csv')

# Convert to Numpy arrays
X = df[['B02','B03','B04','B08']].to_numpy()
y = df[['Gewas']].to_numpy()
print(y)


[[ 0]
 [ 0]
 [ 0]
 ...
 [10]
 [10]
 [10]]


# Define the Keras Model

We create a Sequential model and add layers one at a time until we are happy with our network architecture.

The first thing to get right is to ensure the input layer has the correct number of input features. This can be specified when creating the first layer with the input_shape argument and setting it to (4,) for presenting the four input variables as a vector.

How do we know the number of layers and their types?

This is a tricky question. There are heuristics that you can use, and often the best network structure is found through a process of trial and error experimentation (I explain more about this here). Generally, you need a network large enough to capture the structure of the problem.

In this example, let’s use a fully-connected network structure with three layers.

Fully connected layers are defined using the Dense class. You can specify the number of neurons or nodes in the layer as the first argument and the activation function using the activation argument.

Also, you will use the rectified linear unit activation function referred to as ReLU on the first two layers and the Sigmoid function in the output layer.

It used to be the case that Sigmoid and Tanh activation functions were preferred for all layers. These days, better performance is achieved using the ReLU activation function. Using a sigmoid on the output layer ensures your network output is between 0 and 1 and is easy to map to either a probability of class 1 or snap to a hard classification of either class with a default threshold of 0.5.

You can piece it all together by adding each layer:

- The model expects rows of data with 4 variables (the input_shape=(4,) argument).
- The first hidden layer has 8 nodes and uses the relu activation function.
- The second hidden layer has 4 nodes and uses the relu activation function.
- The output layer has one node and uses the sigmoid activation function.

In [16]:
# define the keras model
model = Sequential()
model.add(Dense(4, input_shape=(4,), activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

## Compiling the model

Compiling the model uses the efficient numerical libraries under the covers (the so-called backend) such as Theano or TensorFlow. The backend automatically chooses the best way to represent the network for training and making predictions to run on your hardware, such as CPU, GPU, or even distributed.

When compiling, you must specify some additional properties required when training the network. Remember training a network means finding the best set of weights to map inputs to outputs in your dataset.

You must specify the loss function to use to evaluate a set of weights, the optimizer used to search through different weights for the network, and any optional metrics you want to collect and report during training.

In this case, use cross entropy as the loss argument. This loss is for a binary classification problems and is defined in Keras as “binary_crossentropy“. You can learn more about choosing loss functions based on your problem [here](https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/)

We will define the optimizer as the efficient stochastic gradient descent algorithm [adam](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/). This is a popular version of gradient descent because it automatically tunes itself and gives good results in a wide range of problems.

Finally, because it is a classification problem, you will collect and report the classification accuracy defined via the metrics argument.

In [17]:
...
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Fit the Keras model

You have defined your model and compiled it to get ready for efficient computation. Now it is time to execute the model on some data. You can train or fit your model on your loaded data by calling the `fit()` function on the model.

Training occurs over epochs, and each [epoch is split into batches](https://machinelearningmastery.com/difference-between-a-batch-and-an-epoch/).

- Epoch: One pass through all of the rows in the training dataset
- Batch: One or more samples considered by the model within an epoch before weights are updated

One epoch comprises one or more batches, based on the chosen batch size, and the model is fit for many epochs.

The training process will run for a fixed number of epochs (iterations) through the dataset that you must specify using the epochs argument. You must also set the number of dataset rows that are considered before the model weights are updated within each epoch, called the batch size, and set using the batch_size argument.

This problem will run for a small number of epochs (150) and use a relatively small batch size of 10.

These configurations can be chosen experimentally by trial and error. You want to train the model enough so that it learns a good (or good enough) mapping of rows of input data to the output classification. The model will always have some error, but the amount of error will level out after some point for a given model configuration. This is called model convergence

In [18]:
...
# fit the keras model on the dataset
model.fit(X, y, epochs=100, batch_size=10)

Epoch 1/100
233/233 [==============================] - 1s 2ms/step - loss: -724.7368 - accuracy: 0.0752
Epoch 2/100
233/233 [==============================] - 0s 2ms/step - loss: -21333.6289 - accuracy: 0.1074
Epoch 3/100
233/233 [==============================] - 0s 2ms/step - loss: -100926.4531 - accuracy: 0.1074
Epoch 4/100
233/233 [==============================] - 0s 2ms/step - loss: -281109.6562 - accuracy: 0.1074
Epoch 5/100
233/233 [==============================] - 0s 2ms/step - loss: -591606.3125 - accuracy: 0.1074
Epoch 6/100
233/233 [==============================] - 0s 2ms/step - loss: -1056063.5000 - accuracy: 0.1074
Epoch 7/100
233/233 [==============================] - 0s 2ms/step - loss: -1689839.7500 - accuracy: 0.1074
Epoch 8/100
233/233 [==============================] - 0s 2ms/step - loss: -2507428.5000 - accuracy: 0.1074
Epoch 9/100
233/233 [==============================] - 0s 2ms/step - loss: -3518403.5000 - accuracy: 0.1074
Epoch 10/100
233/233 [===============

## Evaluate Keras Model

You have trained our neural network on the entire dataset, and you can evaluate the performance of the network on the same dataset.

This will only give you an idea of how well you have modeled the dataset (e.g., train accuracy), but no idea of how well the algorithm might perform on new data. This was done for simplicity, but ideally, you could separate your data into train and test datasets for training and evaluation of your model.

You can evaluate your model on your training dataset using the evaluate() function and pass it the same input and output used to train the model.

This will generate a prediction for each input and output pair and collect scores, including the average loss and any metrics you have configured, such as accuracy.

The `evaluate()` function will return a list with two values. The first will be the loss of the model on the dataset, and the second will be the accuracy of the model on the dataset. You are only interested in reporting the accuracy so ignore the loss value.

In [6]:
# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

73/73 [==============================] - 0s 2ms/step - loss: -157624624.0000 - accuracy: 0.1074
Accuracy: 10.74
